# 제 4고지 : 신경망 만들기

In [1]:
import sys
sys.path.append('..')

from dezero import Variable
from dezero.utils import *
from tools import plot_2dfunc
import dezero.functions as F

import numpy as np
import matplotlib.pyplot as plt

## step38 : 형상 변환 함수

transpose 함수는 단순히 2차원 행렬을 전치시키는 것 뿐만 아니라 형상의 인덱스 위치를 변경할 수 있다.

In [6]:
x = Variable(np.random.randn(2, 3, 4))

y = x.transpose(1, 2, 0)
print(y.shape)
print(y)

(3, 4, 2)
variable([[[-1.01475678 -1.51587318]
           [ 0.4530077   0.31642428]
           [-0.58332599  1.44420605]
           [ 0.5154171  -0.5258632 ]]
         
          [[-0.70220047  1.18337572]
           [ 0.92431689 -0.37869632]
           [ 0.34228827 -0.52144449]
           [ 0.06053645 -0.7238833 ]]
         
          [[ 0.86566651  1.48096632]
           [-0.99976049 -0.93434245]
           [ 1.67412015  0.51155761]
           [-0.43889434 -0.26727164]]])


원소의 개수는 형상의 숫자를 모두 곱한 값과 같다. 원소의 개수를 유지하는 상태에서 형상을 변화시킬 수 있다.

## step40 : 브로드캐스트 함수
> 브로드캐스트란 형상이 다른 다차원 배열끼리의 연산을 가능하게 하는 넘파이 기능이다.

dezero 프레임워크는 넘파이 기반임을 알고는 있었지만 생각보다 많이 의존한다는 생각이 들었다. <br>그러니까 프레임워크의 설계에 중점을 두고 그 외 브로드캐스트의 정확한 구현같은 것들은 넘파이의 기능으로 대체하여 간편하게 설명한다.
이번 단계에서는 브로드캐스팅 기능을 Function 클래스에 속하게 하여서 순전파와 역전파에서 사용 가능하도록 만든다.

함수에서 브로드캐스팅이 일어난다면 **"브로드캐스팅 -> 함수 계산"** 순서일 것이다. 따라서 역전파의 구현 순서는 기존 함수 구조에서 역전파 계산 이후<br>
순선파시의 브로드캐스팅 여부 확인 후 형상 조정, 즉 **"기울기 계산 -> sum_to"** 이다.